In [1]:
%load_ext autoreload
%autoreload 2
from geolifeclef.utils import get_spark
from pyspark.sql import functions as F

spark = get_spark()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/02 20:53:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/02 20:53:01 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [2]:
data_root = "gs://dsgt-clef-geolifeclef-2024/data"

stats = spark.read.parquet(f"{data_root}/processed/geolsh_graph/v1/stats")
stats.printSchema()

root
 |-- summary: string (nullable = true)
 |-- src: string (nullable = true)
 |-- dst: string (nullable = true)
 |-- count: string (nullable = true)
 |-- threshold: integer (nullable = true)



In [7]:
stats.show()

+-------+------------------+------------------+------------------+---------+
|summary|               src|               dst|             count|threshold|
+-------+------------------+------------------+------------------+---------+
|  count|           5167082|           5167082|           5167082|     1000|
|   mean|  5597.42167455442|  5597.42167455442|44.764387520848324|     1000|
| stddev| 3259.514593942091| 3259.514593942115|  972.323680073277|     1000|
|    min|               0.0|               0.0|                 1|     1000|
|    max|           11254.0|           11254.0|           1247736|     1000|
|  count|           4732856|           4732856|           4732856|      500|
|   mean| 5598.413626360067| 5598.413626360067| 42.03220190937565|      500|
| stddev|3261.2980000576767| 3261.298000057616| 899.7202699174672|      500|
|    min|               0.0|               0.0|                 1|      500|
|    max|           11254.0|           11254.0|            969034|      500|

In [6]:
stats.orderBy(F.col("threshold")).where(F.col("summary") == "mean").show()

+-------+-----------------+-----------------+------------------+---------+
|summary|              src|              dst|             count|threshold|
+-------+-----------------+-----------------+------------------+---------+
|   mean|5613.697802659685|5613.697802659685| 27.08141590197082|       10|
|   mean|5600.516940224454|5600.516940224454|31.260099451768376|       50|
|   mean| 5598.44524144804| 5598.44524144804|34.815734863663806|      100|
|   mean|5598.413626360067|5598.413626360067| 42.03220190937565|      500|
|   mean| 5597.42167455442| 5597.42167455442|44.764387520848324|     1000|
|   mean| 5599.02915942107| 5599.02915942107|50.962236879300704|     5000|
+-------+-----------------+-----------------+------------------+---------+



In [10]:
edges = spark.read.parquet(f"{data_root}/processed/geolsh_graph/v1/edges")
edges.printSchema()
edges.count()

root
 |-- src: double (nullable = true)
 |-- dst: double (nullable = true)
 |-- count: long (nullable = true)
 |-- threshold: integer (nullable = true)



26395576

In [11]:
edges.groupBy("threshold").count().orderBy("threshold").show()

+---------+-------+
|threshold|  count|
+---------+-------+
|       10|2664676|
|       50|3464594|
|      100|3829064|
|      500|4732856|
|     1000|5167082|
|     5000|6537304|
+---------+-------+



In [13]:
(
    edges.where("threshold=5000")
    .groupBy("src")
    .agg(F.count("*").alias("n"))
    .groupBy("n")
    .count()
    .describe()
    .show()
)

+-------+------------------+------------------+
|summary|                 n|             count|
+-------+------------------+------------------+
|  count|              2457|              2457|
|   mean|1519.8042328042327|4.2157102157102155|
| stddev|1036.8822451188282|10.896531318839038|
|    min|                 1|                 1|
|    max|              4869|               208|
+-------+------------------+------------------+



5km is actually a pretty reasonable distance to use for the nearest neighbor graph.